# Lab 12: Memory Systems — Short‑Term vs Long‑Term + Conversation Summaries (LangGraph)

**Goal:** Add a summarizer node (short‑term memory housekeeping) and load a user profile (long‑term memory).

### Setup
```bash
pip install -U langgraph langchain langchain-openai typing_extensions tiktoken
export OPENAI_API_KEY="your_key"
```

In [ ]:

from typing import Dict, Tuple, Optional
import re, json, os

from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from typing_extensions import TypedDict, Annotated
from operator import add

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver


### Tools

In [ ]:

@tool("get_weather", return_direct=False)
def get_weather(city: str) -> str:
    """Return a minimal weather string for the given city using a tiny offline map."""
    weather_db: Dict[str, str] = {"paris":"sunny, 24°C","chicago":"cloudy, 18°C","mumbai":"rainy, 30°C","london":"overcast, 17°C","tokyo":"clear, 26°C"}
    c = city.strip().lower()
    return f"The weather in {city} is {weather_db[c]}." if c in weather_db else f"No weather found for '{city}'. Assume mild (22°C) and clear for demo."


In [ ]:

@tool("mini_wiki", return_direct=False)
def mini_wiki(topic: str) -> str:
    """Return a one-sentence fact for a known city from a tiny offline KB."""
    kb: Dict[str, str] = {"paris":"Paris is the capital of France, known for the Eiffel Tower and the Louvre.","london":"London is the capital of the UK, home to the British Museum and the Thames.","tokyo":"Tokyo blends tradition and technology; famous for Shibuya Crossing and Ueno Park.","mumbai":"Mumbai is India's financial hub, known for Marine Drive and film industry.","chicago":"Chicago sits on Lake Michigan; known for the Riverwalk and deep-dish pizza."}
    return kb.get(topic.strip().lower(), "No entry found in mini_wiki.")


In [ ]:

def _parse_city_weather(query: str):
    import re
    q = query.strip()
    if ";" in q or "city=" in q.lower():
        parts = [p.strip() for p in q.split(";")]
        city, weather = "", ""
        for p in parts:
            if "=" in p:
                k, v = p.split("=", 1)
                k = k.strip().lower(); v = v.strip()
                if k == "city": city = v
                elif k == "weather": weather = v
        if not city: city = re.split(r"[;,]", q)[0].replace("city","").replace("=","").strip()
        return city, weather
    return q, ""


In [ ]:

@tool("suggest_city_activities", return_direct=False)
def suggest_city_activities(query: str) -> str:
    """Recommend ONE indoor and ONE outdoor activity for a city."""
    catalog = {"chicago":{"indoor":["Art Institute of Chicago","Museum of Science and Industry","Field Museum"],"outdoor":["Chicago Riverwalk","Millennium Park","Navy Pier"]},"paris":{"indoor":["Louvre Museum","Musée d'Orsay"],"outdoor":["Seine River Walk","Jardin du Luxembourg"]},"london":{"indoor":["British Museum","Tate Modern"],"outdoor":["Hyde Park","South Bank Walk"]},"tokyo":{"indoor":["teamLab Planets","Tokyo National Museum"],"outdoor":["Ueno Park","Shibuya Crossing Walk"]},"mumbai":{"indoor":["Chhatrapati Shivaji Maharaj Vastu Sangrahalaya","Phoenix Mall"],"outdoor":["Marine Drive","Sanjay Gandhi National Park"]}}
    city, weather = _parse_city_weather(query)
    c = city.strip().lower()
    if not c: return "Please provide a city name."
    data = catalog.get(c)
    if not data: return "General: Indoor - museum/aquarium. Outdoor - central park/riverfront."
    w = (weather or "").lower()
    indoor_first = any(k in w for k in ["rain","storm"]) or ("overcast" in w and "cold" in w)
    if indoor_first: indoor, outdoor = data["indoor"][0], data["outdoor"][0]
    elif any(k in w for k in ["sunny","clear"]): outdoor, indoor = data["outdoor"][0], data["indoor"][0]
    else: indoor, outdoor = data["indoor"][0], data["outdoor"][0]
    return f"City: {city}. Indoor: {indoor}. Outdoor: {outdoor}. (Weather-aware heuristics.)"


In [ ]:

@tool("calculator", return_direct=False)
def calculator(expression: str) -> str:
    """Evaluate a simple math expression (digits + + - * / ( ) . %)."""
    import math, re
    if not re.fullmatch(r"[0-9+\-*/(). %\s]+", expression): return "Calculator error: invalid characters."
    try: return str(eval(expression, {"__builtins__": {}}, {"math": math}))
    except Exception as e: return f"Calculator error: {e}"


### Build Graph with Planner, Executor, Summarizer (STM)

In [ ]:

from typing_extensions import TypedDict, Annotated
from operator import add
from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

TOOLS = [get_weather, mini_wiki, suggest_city_activities, calculator]

class MemState(TypedDict):
    messages: Annotated[list[AnyMessage], add]
    plan: dict | None
    exec_result: str | None
    summary: str | None
    turn_count: int
    user_profile: dict | None

planner_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
executor_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
executor_llm_with_tools = executor_llm.bind_tools(TOOLS)
summ_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

PLANNER_SYS = "You are PLANNER. JSON only. Use user preferences from SYSTEM messages."
EXECUTOR_SYS = "You are EXECUTOR. Execute step with tools, or summarize the latest tool result as EXEC_RESULT: <one line>."

def _extract_json(text: str):
    import re, json
    try: return json.loads(text)
    except Exception:
        m = re.search(r"\{.*\}\s*$", text, re.S)
        return json.loads(m.group(0)) if m else {}

def planner_node(state: MemState) -> MemState:
    msgs = state["messages"]
    sys_msgs = []
    if not msgs or msgs[0].type != "system":
        sys_msgs.append(SystemMessage(content=PLANNER_SYS))
        profile = state.get("user_profile") or {}
        if profile:
            prefs = profile.get("preferences", {})
            sys_msgs.append(SystemMessage(content=f"User profile: name={profile.get('name','User')}. Preferences={prefs}"))
        if state.get("summary"):
            sys_msgs.append(SystemMessage(content=f"Conversation summary so far:\n{state['summary']}"))
    response = planner_llm.invoke(sys_msgs + msgs)
    turn = int(state.get("turn_count", 0)) + 1
    return {"messages": [response], "plan": _extract_json(response.content), "turn_count": turn}

def executor_node(state: MemState) -> MemState:
    msgs = list(state["messages"])
    last = msgs[-1]
    if isinstance(last, ToolMessage):
        exec_msgs = msgs + [SystemMessage(content=EXECUTOR_SYS), HumanMessage(content="Summarize as EXEC_RESULT: ...")]
    else:
        step = (state.get("plan") or {}).get("next_step", {})
        exec_msgs = msgs + [SystemMessage(content=EXECUTOR_SYS), HumanMessage(content=f"Step to execute: {json.dumps(step)}")]
    response = executor_llm_with_tools.invoke(exec_msgs)
    return {"messages": [response]}

def summarizer_node(state: MemState) -> MemState:
    msgs = state["messages"]
    recent = []
    for m in msgs[-20:]:
        if m.type in ("human","ai","tool"):
            recent.append(f"[{m.type}] {getattr(m,'content','')}")
    prompt = [SystemMessage(content="Summarize in 5 bullets, keep user prefs and key facts."),
              HumanMessage(content="\n".join(recent) if recent else "No prior content.")]
    summary = summ_llm.invoke(prompt).content.strip()
    return {"messages": [SystemMessage(content=f"Conversation summary so far:\n{summary}")], "summary": summary}

graph = StateGraph(MemState)
graph.add_node("planner", planner_node)
graph.add_node("executor", executor_node)
graph.add_node("tools", ToolNode(TOOLS))
graph.add_node("summarizer", summarizer_node)

def route_from_planner(state: MemState):
    if (state.get("plan") or {}).get("done"): return END
    return "summarizer" if (int(state.get("turn_count",0)) % 3 == 0) else "executor"

def route_from_summarizer(state: MemState): return "executor"
def route_from_executor(state: MemState):
    last = state["messages"][-1]
    tc = getattr(last,"tool_calls",None) or (getattr(last,"additional_kwargs",{}) or {}).get("tool_calls")
    return "tools" if tc else "planner"

graph.set_entry_point("planner")
graph.add_conditional_edges("planner", route_from_planner)
graph.add_conditional_edges("summarizer", route_from_summarizer)
graph.add_conditional_edges("executor", route_from_executor)
graph.add_edge("tools", "executor")

checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)


### Demo: LTM (profile) + STM (summary)

In [ ]:

def load_user_profile():
    return {"name":"Priya","preferences":{"indoor": True, "budget": "low", "likes_museums": True}}

cfg = {"configurable": {"thread_id": "lab12-notebook"}}
state_in = {
    "messages": [HumanMessage(content="Plan a short evening in Paris. Keep my preferences in mind. First weather, then one indoor and one outdoor, then finalize.")]
    , "plan": None, "exec_result": None, "summary": None, "turn_count": 0, "user_profile": load_user_profile()
}
out = app.invoke(state_in, cfg)
out["messages"][-1]
